#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load embedding model into ollama:
    1. Attach ollama container shell
    2. Run command in container `ollama pull pull bge-m3`
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Langchain Ollama Embeddings](https://python.langchain.com/docs/integrations/text_embedding/ollama/)
- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [19]:
from langchain_ollama import OllamaEmbeddings

embedding_model = OllamaEmbeddings(
    model="bge-m3",
)

In [20]:
text = "This is a test document."

query_vector = embedding_model.embed_query(text)

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 1024
[-0.050017744, 0.021129876, -0.057069883, -0.0055166543, -0.0062899087, -0.02841217, 0.045161195, 0.042706154, -0.00025403852, 0.013057328]


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: ``PyPDFLoader``
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [21]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
loader = PyPDFLoader(pdf_doc)

# Load and split documents in chunks
pages_chunked = loader.load_and_split(text_splitter=RecursiveCharacterTextSplitter())

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

pages_chunked_cleaned = [clean_text(chunk.page_content) for chunk in pages_chunked]

print(pages_chunked[0])

page_content='Aurélien GéronHands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent SystemsSECOND EDITION
Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [23]:
print(pages_chunked[1])

page_content='978-1-492-03264-9
[LSI]Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurélien Géron
Copyright © 2019 Aurélien Géron. All rights reserved.
Printed in the United States of America.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles ( http://oreilly.com ). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com .
Editor:  Nicole Tache
Interior Designer:  David FutatoCover Designer:  Karen Montgomery
Illustrator:  Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649  for release details.
The O’Reilly logo 

In [7]:
print(f"Number of text chunks: {len(pages_chunked)}")

Number of text chunks: 507


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)

In [24]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

collection = client.get_or_create_collection("ai_model_book")

vector_db_from_client = Chroma(
    client=client,
    collection_name="ai_model_book",
    embedding_function=embedding_model,
)

In [25]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked[:50]))]

vector_db_from_client.add_documents(documents=pages_chunked[:50], ids=uuids)

['a5cc1801-ab92-4590-ae4a-edfa96466a2f',
 '3a21ae2e-78fe-4fb4-b0eb-a7b26f2f90fa',
 '84a6801f-b0dd-4069-b286-fed2d577f477',
 '576247b4-240c-469a-baab-d48c1a491e03',
 '23928335-1806-4f00-83a6-e07e5d872b45',
 'f317bac0-94b0-480c-8736-5e23d825083c',
 'c94e8651-2fb6-4d9f-ba01-2f5d6a0e7be5',
 '05998f8b-d70a-422c-a209-7c810e14610b',
 '525b05fb-90eb-4d9c-82cc-774e9e140050',
 'c04e9b28-20a8-4f67-95d8-a6569d591901',
 'd04d32af-b13a-4ba5-9925-5f13b5a0e882',
 '51af3226-99c7-4105-89a9-8884a239983a',
 '892a433a-1930-4777-8bd3-0a398addd88b',
 '1b466481-7c35-4ae0-8007-5882496bc58b',
 '656f53e4-80bc-4559-8f66-b3bbfe6d2017',
 'd5ee9d93-1c0b-4566-a3d1-8c6ab9b6a3ec',
 'f8fbc816-6122-4923-9b63-d2d34179f097',
 'ca52d975-7015-4e57-b678-7d70a7dd7bbc',
 '98c5a999-be9d-4d3b-a610-d969741505d1',
 'b7529d5a-533a-40ac-b5da-ae7cf249e7b2',
 'e9fc90f6-2d4e-45b5-ac8d-f0af513580ed',
 'e8ca3566-dac5-4a6c-a04f-068d0cd05096',
 'ac19eeb0-b04e-4aa2-9c82-813a645eb878',
 '60bfaa36-ac45-48ee-b721-5fa39dc7527a',
 'f0b84105-8892-

In [26]:
client.count_collections()

1

In [16]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents


**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)

In [33]:
search_query = "Types of Machine Learning Systems"

results = vector_db_from_client.similarity_search(
    search_query,
    k=3
)
for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

Types of Machine Learning Systems
There are so many different types of Machine Learning systems that it is useful to
classify them in broad categories based on:
•Whether or not they are trained with human supervision (supervised, unsuper‐
vised, semisupervised, and Reinforcement Learning)
•Whether or not they can learn incrementally on the fly (online versus batch
learning)
•Whether they work by simply comparing new data points to known data points,
or instead detect patterns in the training data and build a predictive model, much
like scientists do (instance-based versus model-based learning)
These criteria are not exclusive; you can combine them in any way you like. For
example, a state-of-the-art spam filter may learn on the fly using a deep neural net‐
work model trained using examples of spam and ham; this makes it an online, model-
based, supervised learning system.
Let’s look at each of these criteria a bit more closely.
Supervised/Unsupervised Learning
Machine Learning systems 